<a href="https://colab.research.google.com/github/hijera/NN_colab_hw/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%BE_%D1%82%D0%B5%D0%BC%D0%B5_%C2%AB%D0%9A%D0%BE%D0%BB%D0%BB%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%B0%D1%86%D0%B8%D1%8F%C2%BB_(%D0%92%D0%BB%D0%B0%D1%81%D0%BE%D0%B2_%D0%90%D0%BD%D1%82%D0%BE%D0%BD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Пакет SURPRISE:

1.  используйте данные MovieLens 1M,
2.  можно использовать любые модели из пакета,
3.  получите RMSE на тестовом сете 0,87 и ниже.

Комментарий преподавателя:
В домашнем задании на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете.


In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163758 sha256=e3c78a2ff2a67cd24b7fe0f32a0065339cfaf1967c53c486c76df0e604bd0275
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline

#Импорт surprise
from surprise import KNNWithMeans, KNNBasic,SVD,BaselineOnly
from surprise import NormalPredictor,KNNWithZScore,SVDpp,NMF,SlopeOne,CoClustering
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
%matplotlib inline

In [ ]:
data = Dataset.load_builtin('ml-100k',prompt=False)
#data.__dict__
data_movielens=pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id','item_id','rating','timestamp'])
data_movielens=data_movielens.drop(columns=['timestamp'])
data_movielens

Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


,user_id,item_id,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0
...,...,...,...
99995,880,476,3.0
99996,716,204,5.0
99997,276,1090,1.0
99998,13,225,2.0


In [ ]:
#jdata=ratings.join(movies.set_index('movieId'),on='movieId')
#jdata_scoped=jdata[['userId','title','rating']]
#jdata_scoped

In [ ]:
reader = Reader(rating_scale=(data_movielens.rating.min(), data_movielens.rating.max()))
data = Dataset.load_from_df(data_movielens, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.20)

In [ ]:


algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)
# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

RMSE: 0.9419
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9327  0.9350  0.9391  0.9280  0.9437  0.9357  0.0054  
MAE (testset)     0.7355  0.7387  0.7396  0.7314  0.7438  0.7378  0.0042  
Fit time          4.09    3.13    2.87    1.55    1.92    2.71    0.90    
Test time         0.67    0.47    0.22    0.14    0.40    0.38    0.19    


{'test_rmse': array([0.93267083, 0.93497456, 0.93909232, 0.92800677, 0.94371648]),
 'test_mae': array([0.73546915, 0.73868574, 0.73962317, 0.73144971, 0.7438404 ]),
 'fit_time': (4.086658239364624,
  3.1328725814819336,
  2.868192195892334,
  1.5469801425933838,
  1.9163527488708496),
 'test_time': (0.6722252368927002,
  0.4707951545715332,
  0.22352147102355957,
  0.13867878913879395,
  0.4043862819671631)}

RMSE вышло больше **0.87**, а нам нужно меньше. попробуем другие алгоритмы.

In [ ]:
another_algos=[KNNWithMeans(), KNNBasic(),BaselineOnly(),NormalPredictor(),KNNWithZScore(),SVDpp(),NMF(),SlopeOne(),CoClustering()]
for algo in another_algos:
  print("Algo: "+algo.__class__.__name__)
  algo.fit(trainset)
  predictions = algo.test(testset)

  # Then compute RMSE
  accuracy.rmse(predictions)
  # Run 5-fold cross-validation and print results
  cross_validate(algo, data, measures=['rmse', 'mse', 'mae', 'fcp'], cv=5, verbose=True)
  print(" ")

Algo: KNNWithMeans
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9569
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MSE, MAE, FCP of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9351  0.9516  0.9473  0.9567  0.9600  0.9501  0.0087  
MSE (testset)     0.8743  0.9055  0.8974  0.9152  0.9216  0.9028  0.0165  
MAE (testset)     0.7376  0.7489  0.7447  0.7537  0.7571  0.7484  0.0068  
FCP (testset)     0.7074  0.7003  0.7083  0.7024  0.6977  0.7032  0.0041  
Fit time          0.44    0.54    0.63    0.47    0.61    0.54    0.07    
Test ti

Все варианты больше 0.87. Попробуем GridSearchCV, и поперебираем параметры

In [ ]:
from surprise.model_selection import GridSearchCV

In [ ]:
from surprise.model_selection import GridSearchCV
param_grid = {"n_epochs": [40, 50], "reg_all": [ 0.0005,0.001],"lr_all":[0.0005,0.001],"n_factors":[20,100],"cache_ratings":[True],"random_state":[42]}
gs = GridSearchCV(SVDpp, param_grid, measures=["rmse","mae","mse","fcp"], cv=5,n_jobs=-1,joblib_verbose=6)
#(cache_ratings=True,random_state=42)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

ModuleNotFoundError: ignored

In [ ]:
param_grid = {"k":[10,20,50,70,100],"random_state":[42]} #,"cache_ratings":[True]
gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse","mae"], cv=5,n_jobs=-1,refit=True,joblib_verbose=6) #,"mae","mse","fcp"
#(cache_ratings=True,random_state=42)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   27.5s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  1.4min finished


Computing the msd similarity matrix...
Done computing similarity matrix.
0.9509794517412902
{'k': 50, 'random_state': 42}
